In [1]:
%config IPCompleter.greedy=True

import import_ipynb
from functions import get_upcoming_team_fixtures_data
from functions import get_upcoming_fixtures_data
from functions import get_past_fixtures_data
from functions import get_past_team_fixtures_data

import numpy as np

import os
import requests
import pandas

importing Jupyter notebook from functions.ipynb


In [2]:
import plotly.offline
import plotly.graph_objects as go

import chart_studio
import chart_studio.plotly as py
#chart_studio .tools.set_credentials_file(username='user', api_key='***')

In [4]:
season='2019-20'
base_path = 'C:/Users/antonia/source/antoniaelek/fantasy-premier-league/scraper/'

# Fixtures

In [171]:
def next_fixtures(season, base_path, no_fixtures=6):
    clubs_h = list(get_upcoming_fixtures_data(base_path, season)['team_h'].unique())
    clubs_a = list(get_upcoming_fixtures_data(base_path, season)['team_a'].unique())
    clubs = sorted(set(clubs_h + clubs_a))
    
    fig = go.Figure()
    alldf=pandas.DataFrame()

    for club in clubs:
        df = get_upcoming_team_fixtures_data(club, base_path, season).head(no_fixtures)
        df['team']=club
        df['sum_difficulty'] = df['difficulty'].rolling(min_periods=1, window=no_fixtures).sum()
        alldf = alldf.append(df)
        
    return alldf.set_index(['team'])

In [221]:
fix = next_fixtures(season, base_path, no_fixtures=4)

In [223]:
fix.loc['Chelsea']

,code,event,kickoff_time,is_home,where,opponent,difficulty,difficulty_other,relative_difficulty,sum_difficulty
team,,,,,,,,,,
Chelsea,1059734,4,2019-08-31T14:00:00Z,True,H,Sheffield Utd,2,4,0.333333,2.0
Chelsea,1059751,5,2019-09-14T14:00:00Z,False,A,Wolves,3,4,0.500000,5.0
Chelsea,1059754,6,2019-09-22T15:30:00Z,True,H,Liverpool,4,4,1.000000,9.0
Chelsea,1059764,7,2019-09-28T14:00:00Z,True,H,Brighton,2,4,0.333333,11.0


In [224]:
fix.loc['West Ham']

,code,event,kickoff_time,is_home,where,opponent,difficulty,difficulty_other,relative_difficulty,sum_difficulty
team,,,,,,,,,,
West Ham,1059741,4,2019-08-31T14:00:00Z,True,H,Norwich,2,3,0.5,2.0
West Ham,1059743,5,2019-09-16T19:00:00Z,False,A,Aston Villa,2,2,1.0,4.0
West Ham,1059761,6,2019-09-22T13:00:00Z,True,H,Man Utd,4,3,2.0,8.0
West Ham,1059762,7,2019-09-28T14:00:00Z,False,A,Bournemouth,3,2,2.0,11.0


In [241]:
def next_fixtures_plot(data, limit_diff=100.0):
    clubs_h = list(get_upcoming_fixtures_data(base_path, season)['team_h'].unique())
    clubs_a = list(get_upcoming_fixtures_data(base_path, season)['team_a'].unique())
    clubs = sorted(set(clubs_h + clubs_a))
    
    fig = go.Figure()

    for club in clubs:
        df = data.loc[club]

        if (df.iloc[-1]['sum_difficulty']>limit_diff):
            continue
        
        fig.add_trace(go.Scatter(
            x=df['event'], 
            y=df['difficulty'],
            name=club,          
            mode='lines',
            line=dict(shape='spline', width=4, smoothing=1.3),
            text = df['opponent'] + ' (' + df['where'] + ')',
            hoverlabel= dict(
                font=dict(color='#404040'),
                bordercolor='#404040',
                bgcolor='white'
            ),            
            hovertemplate = "<b>"+club+"</b></br></br>vs %{text}</br></br><extra></extra>"))
        
    fig.update_layout(
        hovermode='x',
        legend=go.layout.Legend(
            traceorder="normal",
            font=dict(color="#eee"),
            bgcolor="rgba(0,0,0,0)"
        )
    )
    
    x_min=df[['event']].iloc[0]-1
    x_max=df[['event']].iloc[-1]+1
    fig.update_yaxes(title_text="Difficulty",color='#eee',showgrid=False, zeroline=False, tick0=1, dtick=1, range=[1.5, 5])
    fig.update_xaxes(title_text="Gameweek",color='#eee',showgrid=True, zeroline=True, tick0=1, dtick=1, range=[x_min, x_max])

    fig.update_scenes(bgcolor='rgba(0,0,0,0)')

    fig.layout.update(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
    chart_studio.plotly.plot(fig, filename="easiest_schedule")
    plotly.offline.iplot(fig)

In [239]:
next_fixtures_plot(fix,limit_diff=10)

# Form

In [93]:
def form(season, base_path, no_fixtures=6):
    clubs_h = list(get_upcoming_fixtures_data(base_path, season)['team_h'].unique())
    clubs_a = list(get_upcoming_fixtures_data(base_path, season)['team_a'].unique())
    clubs = sorted(set(clubs_h + clubs_a))

    alldf=pandas.DataFrame()
    
    j=0
    for club in clubs:
        df = get_past_team_fixtures_data(club,base_path,season).tail(no_fixtures)
        df['team']=club
        df['points']=df.apply(points, axis=1)
        df['fixture_form']=(df['scored']-df['concieved']+1)*df['relative_difficulty']*0.33
        df['i']=[1 + x * 0.1 for x in range(0,no_fixtures)]
        df['form']=df['i']*df['fixture_form']
        df['form'] = df['form'].rolling(min_periods=1, window=no_fixtures).sum()
        j+=1
                
        alldf = alldf.append(df)
        
    return alldf.set_index(['team'])


def points(row):
    points = 3 if row.scored > row.concieved else 0
    points = 0 if row.scored < row.concieved else points
    points = 1 if row.scored == row.concieved else points
    
    if (row.difficulty < row.difficulty_other):
        points -= (3-points)
    elif (row.difficulty > row.difficulty_other):
        points += (3-points)
    
    return points

In [94]:
df_form = form(season, base_path,no_fixtures=3)

In [97]:
df_form.loc['Chelsea']

,code,event,kickoff_time,opponent,is_home,where,difficulty,difficulty_other,relative_difficulty,scored,concieved,points,fixture_form,i,form
team,,,,,,,,,,,,,,,
Chelsea,1059710,1,2019-08-11T15:30:00Z,Man Utd,False,A,4,4,1.000000,0.0,4.0,0,-0.990,1.0,-0.9900
Chelsea,1059715,2,2019-08-18T15:30:00Z,Leicester,True,H,3,4,0.500000,1.0,1.0,-1,0.165,1.1,-0.8085
Chelsea,1059727,3,2019-08-24T11:30:00Z,Norwich,False,A,2,4,0.333333,3.0,2.0,3,0.220,1.2,-0.5445


In [113]:
df_form.loc['Newcastle']

,code,event,kickoff_time,opponent,is_home,where,difficulty,difficulty_other,relative_difficulty,scored,concieved,points,fixture_form,i,form
team,,,,,,,,,,,,,,,
Newcastle,1059711,1,2019-08-11T13:00:00Z,Arsenal,True,H,4,3,2.0,0.0,1.0,3,0.00,1.0,0.000
Newcastle,1059718,2,2019-08-17T14:00:00Z,Norwich,False,A,2,2,1.0,1.0,3.0,0,-0.33,1.1,-0.363
Newcastle,1059729,3,2019-08-25T15:30:00Z,Spurs,False,A,4,2,3.0,1.0,0.0,3,1.98,1.2,2.013


In [164]:
def form_plot(data, no_fixtures=6,limit_form=-10):
    clubs_h = list(get_upcoming_fixtures_data(base_path, season)['team_h'].unique())
    clubs_a = list(get_upcoming_fixtures_data(base_path, season)['team_a'].unique())
    clubs = sorted(set(clubs_h + clubs_a))
    
    fig = go.Figure()
    alldf=pandas.DataFrame()
    
    j=0
    for club in clubs:
        df = data.loc[club]
        
        if (df['form'].iloc[-1] < limit_form):
            continue        
        
        alldf = alldf.append(df)
        
        fig.add_trace(go.Scatter(
            x=df['event'], 
            y=df['form'],
            name=club,
            text = df['opponent'],     
            mode='lines+markers',
            line=dict(shape='spline', width=4, smoothing=1.3),
            hoverlabel= dict(font=dict(color='#404040'),
                             bordercolor='#404040',
                             bgcolor='white'
            ),
            hovertemplate = "<b>"+club+"</b></br></br></br>Form: %{y}</br>vs %{text}</br></br><extra></extra>"))
        
    fig.update_layout(
        hovermode='closest',
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        legend=go.layout.Legend(
            traceorder="normal",
            font=dict(color="#eee"),
            bgcolor="rgba(0,0,0,0)"
        )
    )
    
    fig.update_yaxes(title_text="Form",color='#eee',tick0=1,dtick=1,showgrid=False,zeroline=False)
    fig.update_xaxes(title_text="Gameweek",color='#eee',tick0=1,dtick=1,showgrid=True,zeroline=True)

    fig.update_scenes(bgcolor='rgba(0,0,0,0)')

    fig.layout.update(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
    chart_studio.plotly.plot(fig, filename="club_form")
    plotly.offline.iplot(fig)

In [165]:
form_plot(df_form, limit_form=1.5)